# <font color='blue'>Uni-Facef - Pyspark - Parte 1 </font>

#### Instalando o pacote PySpark em nosso "Cluster de uma Máquina Só rs"

Lembre-se que para utilização do pacote PySpark é necessário fazer a instalação do Apache Spark na máquina 

In [ ]:
!pip install pyspark

In [ ]:
!java -version

#### Criação um sessão no Spark para no Aplicação

In [ ]:
# coding: utf-8
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .appName('ReceitasGov2020') \
    .getOrCreate()

type(spark)

#### Copiando uma arquivo para o Google Colab Lab 

In [ ]:
from google.colab import files
uploaded = files.upload()

#### Criação de um dataframe

Um DataFrame é um conjunto de dados organizado em colunas nomeadas. É conceitualmente equivalente a uma tabela em um banco de dados relacional ou um quadro de dados em R / Python, mas com otimizações mais ricas sob o capô. Os DataFrames podem ser construídos a partir de uma ampla variedade de fontes 

In [ ]:
df = spark.read \
    .option("delimiter", ";") \
    .csv('Receitas2020.csv', header=True)

type(df)

In [ ]:
# Listando os métodos do objeto DataFrame
dir(df)

In [ ]:
# Conta a quantidade de registros do Dataframe
df.count()

In [ ]:
# Mostra a descrição do Método
help(df.count)

In [ ]:
# Mostra a Schema Inferido para o Dataframa
df.printSchema()

In [ ]:
# Mostra os primeiros registros do dataset (default 20)
df.show(5)

In [ ]:
# Lista para os novos nomes
novos_nomes = [
    "cod_orgao_superior", "nome_orgao_superior", "cod_orgao",
    "nome_orgao", "cod_unidade_gestora", "nome_unidade_gestora",
    "categoria_economica", "origem_receita", "especie_receita",
    "detalhamento", "vr_previsto_atualiz", "vr_lancado",
    "vr_realizado", "percent_realizado", "data_lancamento",
    "ano_exercicio"
]

for i, coluna in enumerate(df.columns):
    print(i, coluna)
    
print("\n1 " + novos_nomes[1])

#### Aplicando alteração no nome dos campos através de funções do módulo "pyspark.sql.functions"

- col() - Para selecionar a coluna que vamos utilizar
- alias() - Para modificar o nome do campo

Obs: A função .upper() e enumerate() são funções do Python 

In [ ]:
import pyspark.sql.functions as sf

df = df.select(
    [sf.col(coluna).alias(novos_nomes[i].upper()) for i, coluna in enumerate(df.columns)]
)
df.printSchema()

#### Criando schema com o módulo "pyspark.sql.types"

É possível definir nome e tipo das colunas do Dataframe, se é opcional ou não, no momento da leitura.

In [ ]:
import pyspark.sql.types as st

SCHEMA = st.StructType([
    st.StructField("cod_orgao_superior", st.LongType(), True),
    st.StructField("nome_orgao_superior", st.StringType(), True),
    st.StructField("cod_orgao", st.LongType(), True),
    st.StructField("nome_orgao", st.StringType(), True),
    st.StructField("cod_unidade_gestora", st.LongType(), True),
    st.StructField("nome_unidade_gestora", st.StringType(), True),
    st.StructField("categoria_economica", st.StringType(), True),
    st.StructField("origem_receita", st.StringType(), True),
    st.StructField("especie_receita", st.StringType(), True),
    st.StructField("detalhamento", st.StringType(), True),
    st.StructField("vr_previsto_atualiz", st.DecimalType(10, 2), True),
    st.StructField("vr_lancado", st.DecimalType(10, 2), True),
    st.StructField("vr_realizado", st.DecimalType(10, 2), True),
    st.StructField("percent_realizado", st.DecimalType(10, 2), True),
    st.StructField("data_lancamento", st.StringType(), True),
    st.StructField("ano_exercicio", st.IntegerType(), True),
    ])

#### Nova leitura do Dataset resolvendo problema do Schema e Coding

In [ ]:
df = spark.read \
    .option("delimiter", ";") \
    .option("encoding", "ISO-8859-1") \
    .csv('Receitas2020.csv', header=True, schema=SCHEMA)

# O parâmetro truncate define se mostra tudo conteudo do campo ou não
df.show(5, truncate=True)

In [ ]:
df.printSchema()

#### Transformando valores através do modulo "pyspark.sql.functions"

- to_timestamp(): Converte o valor string em timestamp, passando o formato atual da string
- cast(): Converte o valor para o type desejado conforme "pyspark.sql.types"

In [ ]:
import pyspark.sql.functions as sf
import pyspark.sql.types as st

df = df \
    .withColumn("data_lancamento", sf.to_timestamp(sf.col("data_lancamento"),"dd/MM/yyyy")) \
    .withColumn("data_lancamento", sf.col("data_lancamento").cast(st.DateType())) \
    #.withColumn("data_lancamento", sf.col("data_lancamento").cast("date")) \

df.select("data_lancamento").show()

In [ ]:
df.printSchema()

In [ ]:
# Mostrando o conteúdo do campo - Valores acentuados
df.select("detalhamento").show(truncate=False)

#### Criando uma UDF (User-defined function)

Criando uma função simples em Python é possível registrá-la como um UDFs e trabalhar da mesma forma como um função do pacote "pyspark.sql.functions" 

In [ ]:
from unicodedata import normalize, category

def remove_acento(texto):
    """
    Remove acentos de um string:
    Essa função utiliza os métodos "normalize" e "category" do módulo "unicodedata"
    Também utiliza "list comprehension" para iterar nos caracteres da string
    """
    if not texto:
        return None
    
    return ''.join((c for c in normalize('NFD', texto) if category(c) != 'Mn'))

# Registra a UDF
remove_acento_udf = sf.udf(remove_acento)

#### Realizando transformações nos campos desejados, usando a UDF "remove_acento_udf"

In [ ]:
df = df \
    .withColumn("nome_orgao_superior", remove_acento_udf("nome_orgao_superior")) \
    .withColumn("nome_orgao", remove_acento_udf("nome_orgao")) \
    .withColumn("nome_unidade_gestora", remove_acento_udf("nome_unidade_gestora")) \
    .withColumn("categoria_economica", remove_acento_udf("categoria_economica")) \
    .withColumn("origem_receita", remove_acento_udf("origem_receita")) \
    .withColumn("especie_receita", remove_acento_udf("especie_receita")) \
    .withColumn("detalhamento", remove_acento_udf("detalhamento"))

df.select("detalhamento").show(truncate=False)

In [ ]:
# Escreve em formato parquet
df.repartition(2).write.parquet(
    "Receitas2020", 
    mode="overwrite")

In [ ]:
!pwd

In [ ]:
!ls -l /home/al_morais/Documentos/UniFacef/notebooks/Receitas2020

### FIM
###### Documentação: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html